<a href="https://colab.research.google.com/github/tc19r/FusedNet/blob/main/test0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
from sklearn.mixture import GaussianMixture
import math

tf.keras.backend.set_floatx('float64')



from tensorflow import keras
import numpy as np
#from my_classes import DataGenerator
import numpy as np
import os
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import  TensorBoard
import matplotlib
matplotlib.use('pdf')
from matplotlib.backends.backend_pdf import PdfPages

from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import numpy as np
from numpy import linalg as LA
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.python.keras.layers import concatenate
import tensorflow as tf
from matplotlib.backends.backend_pdf import PdfPages


#Hyper parameters 
learningrate,lambda1,lambda2=0.0003,1.5e-2,3e-4
epochs = 50
rootpath="./"
num_classes=400
Centers_Lengths_dimension=4
fibs_input_dim=30
dim_fib=30
dim_ctnl=4
recon_dim=fibs_input_dim

#Data
#Note that walk will search subdirectories too.
rootpath='./'
X=np.empty([0,34])
for root, dirs, files in os.walk(rootpath):
     for filename in [fi for fi in files if fi.endswith(".pf")]:
        pf=np.fromfile(filename)
        pf=pf.reshape(int(len(pf)/34),34)
        np.random.shuffle(pf)
        X=np.vstack([X,pf])
        print(pf.shape)
trainset=np.float64(X)

#
x_train=trainset
y_train=trainset[:,0:dim_fib]

batch_size=int(np.floor(trainset.shape[0]/10))
#Model&Trainning


inputs_center_and_length = tf.keras.Input(shape=(Centers_Lengths_dimension,))
inputs_fibs_on_ball = tf.keras.Input(shape=(fibs_input_dim,))

encoding_dim = 4




#define loss.
def getmodel():
    encoder=keras.layers.Dense(16, activation='tanh')(inputs_fibs_on_ball)
    encoder=keras.layers.Dense(16, activation='tanh')(encoder)
    encoder=keras.layers.Dense(8, activation='tanh')(encoder)
    encoder=keras.layers.Dense(4, activation='tanh')(encoder)
    encoder=keras.layers.Dense(encoding_dim, activation='tanh',name="feature")(encoder)
    encoder=keras.Model(inputs=inputs_fibs_on_ball, outputs=encoder)
    #define decoder network:

    decoder=keras.layers.Dense(4,activation='tanh')(encoder.outputs[0])
    decoder=keras.layers.Dense(8, activation='tanh')(decoder)
    #decoder = keras.layers.Dropout(0.25)(decoder)
    decoder=keras.layers.Dense(16,activation='tanh')(decoder)
    #decoder = keras.layers.Dropout(0.25)(decoder)
    decoder=keras.layers.Dense(16,activation='tanh')(decoder)
    decoder=keras.layers.Dense(recon_dim,activation='linear')(decoder)
    decoder=keras.Model(inputs=encoder.inputs, outputs=decoder)
    #===
    combined = concatenate([decoder.output, encoder.output,inputs_center_and_length,],axis=1)
    fusednet = keras.Model(inputs=[inputs_fibs_on_ball,inputs_center_and_length],outputs=combined)
    feature_extractor = keras.Model(
       inputs=inputs_fibs_on_ball,
       outputs=fusednet.get_layer(name="feature").output,
    )
    return fusednet,feature_extractor,
fusednet,feature_extractor = getmodel()
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.batch(batch_size)


import time
model=fusednet
optimizer = tf.keras.optimizers.Adam(learning_rate=learningrate)
#initialize gmm object
gm = GaussianMixture(n_components=num_classes, random_state=0,warm_start=True,max_iter=10)

import time
model=fusednet
optimizer = tf.keras.optimizers.Adam(learning_rate=learningrate)
lr=10000
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()
    
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            y_pred = model([x_batch_train[:,0:dim_fib],x_batch_train[:,dim_fib:dim_fib+dim_ctnl]], training=True)
           #loss_value = loss_fn(y_batch_train, logits)
            recon = y_pred[:,0:recon_dim]
            z = y_pred[:,recon_dim:]
            gm.fit(z)
            print("fitting complete")
            j=0
            l=j*lr 
            r=np.minimum((j+1)*lr,batch_size)
            gamma = gm.predict_proba(z[l:r,:])
            gamma_sum = tf.reduce_sum(gamma, axis=0)
            
            for j in range(1,int(np.ceil(batch_size/lr))):
                l=j*lr 
                r=np.minimum((j+1)*lr,batch_size)
                gamma = gm.predict_proba(z[l:r,:])
                gamma_sum += tf.reduce_sum(gamma, axis=0)
                #phi = tf.reduce_mean(gamma, axis=0)
            phi = gamma_sum/batch_size
            mu = gm.means_
            Sigma = gm.covariances_
            n_features = z.shape[1]
            min_vals = tf.linalg.diag(tf.ones(n_features, dtype=tf.float64)) * 1e-6
            L = tf.linalg.cholesky(Sigma + min_vals[None, :, :])
            z_centered = z[:, None, :] - mu[None, :, :]
            v = tf.linalg.triangular_solve(L, tf.transpose(z_centered, [1, 2, 0]))
            log_det_sigma = 2.0 * tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L)), axis=1)
            d = z.shape[1]
            logits = tf.dtypes.cast(tf.math.log(phi[:, None]), dtype=tf.float64) - 0.5 * (
                    tf.dtypes.cast(tf.reduce_sum(tf.square(v), axis=1), dtype=tf.float64) + tf.dtypes.cast(
                d * tf.math.log(2.0 * np.pi), dtype=tf.float64) + tf.dtypes.cast(log_det_sigma[:, None],
                                                                                 dtype=tf.float64))
            energies = - tf.reduce_logsumexp(logits, axis=0)
            enerloss = tf.reduce_mean(energies)
            input = y_batch_train[:, 0:recon_dim]
            reconloss = tf.math.reduce_sum(tf.reduce_mean(tf.square(recon - input), axis=-1))
            loss_value=lambda1*enerloss + reconloss
            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))


            if step % 10 == 0:
                print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
                )
                print("Seen so far: %s samples" % ((step + 1) * batch_size))
                #print(phi)
    print("Time taken: %.2fs" % (time.time() - start_time))

fusednet.save("./model.h5")
import pickle

with open('gm.pkl','wb') as f:
    pickle.dump(gm,f)
